In [ ]:
import os
import utils
import connectome_create
# viz_method = one of ['itkwidgets', 'vtk']
viz_method = 'vtk'

# import some of our favorite packages
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings

%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
# import nglui.statebuilder as ngstbld

# # this is the EM specific package for querying the EM data
from caveclient import CAVEclient


# from meshparty import trimesh_io, trimesh_vtk
# from meshparty import skeletonize, skeleton_io, skeleton
# import cloudvolume

%load_ext autoreload
%autoreload 2

In [ ]:
# from pathlib import Path
# import json

# with open(Path.home() / '.cloudvolume/secrets/'/'cave-secret.json') as f:
#         tokens = json.load(f)
        
# seg_source = 'graphene://https://cave.fanc-fly.com/segmentation/table/mar2021_prod'
# cv = cloudvolume.CloudVolume(cloudpath=seg_source, use_https=True, secrets=tokens)

In [ ]:
datastack_name = 'fanc_production_mar2021'

client = CAVEclient(datastack_name)

In [ ]:
# client = CAVEclient()

# # if not os.path.isfile(os.path.expanduser("~/.cloudvolume/secrets/cave-secret.json")):
# client.auth.get_new_token(open=True)

In [ ]:
# if you have not yet setup this computer, uncomment this below line
# paste the token from the website in, and run the line

# client.auth.save_token(token="c14cd7a3e18a1a697716a399afbf5778", overwrite=True)

# then comment or delete the line as you don't need to run it on this computer  again

# Load premotor neuron tables

In [ ]:
try:
    pre_to_mn_df = connectome_create.load_pre_to_mn_df(ext='matched_typed_with_nt')
except FileNotFoundError:
    print('Looking for an older table')
    pre_to_mn_df = connectome_create.load_old_pre_to_mn_df(ext='matched_typed_with_nt')
print(pre_to_mn_df.shape)

pre_to_pre_df = connectome_create.load_pre_to_df(ext='_no_index')
pre_to_pre_df.shape

In [ ]:
pre_to_mn_df

# Add the premn index to the premotor df columns!
I would like to make this df wider than it is long, which matches the input to output type layout of pre_to_mn_df, but I think that's less efficient. So keeping it longer than it is wide until i whittle it down.

In [ ]:
idx_df = pre_to_mn_df.index.to_frame()

# Reorder pre_to_pre by the segid
pre_to_pre_df = pre_to_pre_df.loc[:,idx_df.segID]
print(pre_to_pre_df.columns.values[0:15])
print(idx_df.segID.values[0:15])

In [ ]:
pre_to_pre_df.columns = pre_to_mn_df.index
pre_to_pre_df.head()

# Create index multiindex (post- premotor neurons)

In [ ]:
timestamp = connectome_create.get_timestamp()

soma_table = client.materialize.query_table('soma_jan2022',timestamp=timestamp)
all_mns = client.materialize.query_table('all_leg_motor_neuron_table_v0',timestamp=timestamp)
neckns_df = client.materialize.query_table('neck_connective_tag_table_v1',timestamp=timestamp)
all_sensory = connectome_create.get_unduplicated_sensory_axon_table(client)

# Get lists of segIDs to search
motorsids = sorted(pre_to_mn_df.columns.get_level_values('segID').to_list())
premotorsids = sorted(pre_to_mn_df.index.get_level_values('segID').to_list())
localsids = sorted(pre_to_mn_df.loc['local'].index.get_level_values('segID').to_list())
intrsids = sorted(pre_to_mn_df.loc['intersegmental'].index.get_level_values('segID').to_list())
somasids = sorted(soma_table.pt_root_id.to_list())
sensorysids = sorted(all_sensory.pt_root_id.to_list())
necksids = sorted(neckns_df.pt_root_id.to_list())

nont1_mns = all_mns[~all_mns.pt_root_id.isin(pre_to_mn_df.columns.get_level_values('segID'))]
nont1mnsids = sorted(nont1_mns.pt_root_id.to_list())
print('{} T1 mns, {} non T1_mns, {} total'.format(len(motorsids),len(nont1mnsids),all_mns.shape[0]))

In [ ]:
# Now create an index for the parallel neurons
data = np.full((pre_to_pre_df.shape[0], 8), False)
row_idx = pd.DataFrame(data=data,index=pre_to_pre_df.index.to_list(),columns=['motor','has_soma','sensory','neck','local','intersegmental','premotor','non_t1_motor'])
print(row_idx.shape)
row_idx.head()

In [ ]:
row_idx.motor = row_idx.index.isin(motorsids)
row_idx.has_soma = row_idx.index.isin(somasids)
row_idx.sensory = row_idx.index.isin(sensorysids)
row_idx.neck = row_idx.index.isin(necksids)
row_idx.local = row_idx.index.isin(localsids)
row_idx.intersegmental = row_idx.index.isin(intrsids)
row_idx.premotor = row_idx.index.isin(premotorsids)
row_idx.non_t1_motor = row_idx.index.isin(nont1mnsids)

row_idx.head()

# Takes no time now!

In [ ]:
print('{} of {} total non T1L MNs are downstream of T1L premotor neurons.'.format(row_idx.non_t1_motor.sum(),len(nont1mnsids)))

In [ ]:
row_idx['post_pt_root_id'] = row_idx.index
row_idx = row_idx[['post_pt_root_id','motor','has_soma','sensory','neck','local','intersegmental','premotor','non_t1_motor']]
row_idx

In [ ]:
# Put the new index on the pre_to_pre_df
# pre_to_pre_df.index = pd.MultiIndex.from_frame(row_idx)
# print(pre_to_pre_df.shape)
# pre_to_pre_df.head()

# Replace the logical indices with cell_class, premotor
Work with the row_index instead of the giant matrix

In [ ]:
All = slice(None)

# post-pMN knives
# FrozenList(['pt_root_id', 'motor', 'has_soma', 'sensory', 'neck', 'premotor', 'local', 'intrseg'])
# pt_root_id,motor_t1,motor_nont1,has_soma,sensory,neck,premotor,local
# All,motor_t1,motor_nont1,has_soma,sensory,neck,premotor,local,intrseg
print(row_idx.columns)
# 'post_pt_root_id', 'motor', 'has_soma', 'sensory', 'neck', 'local',  'intersegmental', 'premotor', 'non_t1_motor'],

In [ ]:
row_idx.index = pd.MultiIndex.from_frame(row_idx)
row_idx.loc[:,'cell_class'] = np.nan
row_idx = row_idx.drop(['motor', 'has_soma', 'sensory', 'neck', 'local', 'intersegmental', 'premotor', 'non_t1_motor'],axis=1)
row_idx

In [ ]:
cumtargetsum = 0
total_pmn_targets = row_idx.shape[0]

# T1L MNs 
#           [pid, motor,  soma,   sens,   neck,   locl,   intr,   prem,   nont1
m_r_tup =   (All, True,   True,   All,  False,  False,  False,  False,  False)
row_idx.loc[m_r_tup,'cell_class'] = 'motor'
print('{} T1 motor neurons'.format(row_idx.loc[m_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[m_r_tup,:].shape[0]


# other MNs in other segments row tuple
#                   [pid, motor,  soma,   sens,   neck,   locl,   intr,   prem,   nont1
mn_non_t1_r_tup =   (All, False,  True,   All,    False,  False,  False,  False,  True)
row_idx.loc[mn_non_t1_r_tup,'cell_class'] = 'non_t1_motor'
print('{} recipient motor neurons'.format(row_idx.loc[mn_non_t1_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[mn_non_t1_r_tup,:].shape[0]

#                   [pid, motor,  soma,   sens,   neck,   locl,   intr,   prem,   nont1
pmn_sens_r_tup =     (All, False,  True,   True,   False,  False,  False,  True,  False)
print('{} recipient and motor neurons with nerve annotations that are not in motor neuron table'.format(row_idx.loc[pmn_sens_r_tup,:].shape[0]))
row_idx.loc[pmn_sens_r_tup,'cell_class'] = 'non_t1_motor'
cumtargetsum = cumtargetsum + row_idx.loc[pmn_sens_r_tup,:].shape[0]


# strange neuromodulatory ascending neuron
#                   [pid, motor,  soma,   sens,   neck,   locl,   intr,   prem,   nont1
n_r_tup =           (All, False,  True,   True,   True,  False,  False,  False,  False)
row_idx.loc[n_r_tup,'cell_class'] = 'neurmodulatory_ascending_neuron'
print('{} modulatory neurons'.format(row_idx.loc[n_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[n_r_tup,:].shape[0]


# descending, pre and _post, row tuple
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
d_r_tup =           (All, False,  False,    False,  True,   False,  False,  True,  False)
row_idx.loc[d_r_tup,'cell_class'] = 'descending'
print('{} recipient descending neurons that are also premotor'.format(row_idx.loc[d_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[d_r_tup,:].shape[0]

#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
d_r_tup =           (All, False,  False,    False,  True,   False,  False,  False,  False)
row_idx.loc[d_r_tup,'cell_class'] = 'descending_post'
print('{} recipient descending neurons that are not premotor'.format(row_idx.loc[d_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[d_r_tup,:].shape[0]


# ascending, pre and _post, row tuple
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
a_r_tup =           (All, False,  True,     False,  True,   False,  False,  True,    False)
row_idx.loc[a_r_tup,'cell_class'] = 'ascending'
cumtargetsum = cumtargetsum + row_idx.loc[a_r_tup,:].shape[0]
print('{} ascending neurons'.format(row_idx.loc[a_r_tup,:].shape[0]))

a_r_tup =           (All, False,  True,     False,  True,   False,  False,  False,    False)
row_idx.loc[a_r_tup,'cell_class'] = 'ascending_post'
cumtargetsum = cumtargetsum + row_idx.loc[a_r_tup,:].shape[0]
print('{} ascending_post neurons'.format(row_idx.loc[a_r_tup,:].shape[0]))


# sensory, pre and _post, row tuple
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
s_r_tup =           (All, False,  False,    True,   False,  False,  False,  True,    False)
row_idx.loc[s_r_tup,'cell_class'] = 'sensory'
print('{} sensory neurons'.format(row_idx.loc[s_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[s_r_tup,:].shape[0]

#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
s_r_tup =           (All, False,  False,    True,   False,  False,  False,  False,    False)
row_idx.loc[s_r_tup,'cell_class'] = 'sensory_post'
print('{} sensory_post neurons'.format(row_idx.loc[s_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[s_r_tup,:].shape[0]

# local vnc premotor row tuple 
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
# lp_r_tup =           (All, False,  True,    False,  False,  True,   False,  True,   False)
# print('{} local recipient neurons'.format(pre_to_pre_df.loc[lp_r_tup,:].shape[0]))
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
lp_r_tup =           (All, False,  True,    False,  False,  True,   False,  True,   False)
lp_r_tup =           (All, False,  All,     False,  False,  True,   False,  True,   False)  # There a few local neurons with no soma
row_idx.loc[lp_r_tup,'cell_class'] = 'local'
print('{} local recipient neurons'.format(row_idx.loc[lp_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[lp_r_tup,:].shape[0]

# intrsg vnc premotor row tuple 
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
ip_r_tup =          (All, False,  True,    False,  False,  False,   True,  True,   False)
row_idx.loc[ip_r_tup,'cell_class'] = 'intersegmental'
print('{} intersegmental recipient neurons'.format(row_idx.loc[ip_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[ip_r_tup,:].shape[0]

# vnc, but not premotor, row tuple (might be local or intersegmental)
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
vnp_r_tup =          (All, False,  True,    False,  False,  False,  False,  False,   False)
row_idx.loc[vnp_r_tup,'cell_class'] = 'vnc_non_premotor'
print('{} vnc recipient neurons'.format(row_idx.loc[vnp_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[vnp_r_tup,:].shape[0]

# fragments
#                   [pid, motor,  soma,     sens,   neck,   locl,   intr,   prem,   nont1
f_r_tup =           (All, False,  False,    False,  False,  False,   False,  False,   False)
row_idx.loc[f_r_tup,'cell_class'] = 'fragment'
print('{} recipient fragments'.format(row_idx.loc[f_r_tup,:].shape[0]))
cumtargetsum = cumtargetsum + row_idx.loc[f_r_tup,:].shape[0]

# The rest have somas, but also have things in the nerve, i.e. false merges
fm_r_tup =           (All, False,  True,    True,  False,  False,   False,  False,   False)
print('{} false merges'.format(row_idx.loc[fm_r_tup,:].shape[0]))
row_idx.loc[fm_r_tup,'cell_class'] = 'falsemerge'
cumtargetsum = cumtargetsum + row_idx.loc[fm_r_tup,:].shape[0]

print('{} of {} targets unaccounted for'.format(total_pmn_targets-cumtargetsum,total_pmn_targets))

row_idx.loc[row_idx.cell_class.isna()]

In [ ]:
row_idx

In [ ]:
# Nothing has been done to reorder columns, but I want them with mns first, then the same order of the premotor neurons
# First, reorder the columns of row_idx
row_idx = row_idx[['cell_class','post_pt_root_id']]
row_idx

In [ ]:
pre_to_pre_df.index = pd.MultiIndex.from_frame(row_idx)
pre_to_pre_df.groupby(level=[0]).size()

In [ ]:
# put the fragments at the bottom
pre_to_pre_df =  pre_to_pre_df.reindex(axis='index',level=0,labels=['motor','descending','sensory','ascending','intersegmental','local','non_t1_motor','descending_post','sensory_post','ascending_post','neurmodulatory_ascending_neuron','vnc_non_premotor','falsemerge','fragment'])
print(pre_to_pre_df.shape)


In [ ]:
# Save the matrix
connectome_create.save_pre_to_df(pre_to_pre_df,ext='with_class')
# ~3GB

In [ ]:
df_no_frags = pre_to_pre_df.loc[['motor','descending','sensory','ascending','intersegmental','local','non_t1_motor','descending_post','sensory_post','ascending_post','neurmodulatory_ascending_neuron','vnc_non_premotor']]

# Save the matrix
connectome_create.save_pre_to_df(df_no_frags,ext='with_class_no_frag')